In [1]:
import pandas as pd
fin_stat_meta_csv = "financial_statements_metadata_3_09_2025_with_ID - financial_statements_metadata_3_09_2025 - financial_statements_metadata_3_09_2025 (1).csv"
fin_stat_meta_df = pd.read_csv(fin_stat_meta_csv)
fin_stat_meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10113 entries, 0 to 10112
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   symbol              10113 non-null  object 
 1   statement_type      10113 non-null  object 
 2   period              10113 non-null  object 
 3   period_detail       10113 non-null  object 
 4   period_end_date     10113 non-null  object 
 5   report_type         10113 non-null  object 
 6   consolidation_type  10113 non-null  object 
 7   status              1113 non-null   float64
 8   s3_path             8844 non-null   object 
 9   pdf_folder_path     10113 non-null  object 
 10  statement_ID        10113 non-null  object 
 11  pdf_s3_path         9952 non-null   object 
 12  is_missing          10113 non-null  bool   
 13  is_mismatched       10113 non-null  bool   
 14  is_incomplete       10113 non-null  bool   
 15  is_incomplete_note  1126 non-null   object 
 16  is_m

In [2]:

# First, identify which s3_paths are duplicates (appear more than once)
duplicate_s3_paths = fin_stat_meta_df['s3_path'].value_counts()
duplicate_s3_paths = duplicate_s3_paths[duplicate_s3_paths > 1].index
print(f"The number of duplicate s3_paths is {len(duplicate_s3_paths)}")

# Filter the dataframe to only rows with duplicate s3_paths
df_duplicates = fin_stat_meta_df[fin_stat_meta_df['s3_path'].isin(duplicate_s3_paths)]

# Count how many duplicate rows exist per status
duplicate_count_by_status = df_duplicates.groupby('status').size()
print(duplicate_count_by_status)
df_duplicates.to_csv("df_duplicates.csv", index=False)

# Count how many duplicate rows exist per statement_type
duplicate_count_by_statement_type = df_duplicates.groupby('statement_type').size()
print(duplicate_count_by_statement_type)

# Count how many duplicate rows exist per report_type


The number of duplicate s3_paths is 488
status
1.0    16
dtype: int64
statement_type
audited      488
unaudited    488
dtype: int64


In [3]:
# Step 1: Identify which s3_paths are duplicated
duplicate_s3_paths = fin_stat_meta_df['s3_path'].value_counts()
duplicate_s3_paths = duplicate_s3_paths[duplicate_s3_paths > 1].index

# Step 2: Create a boolean mask for rows to keep
# Keep a row if:
# - Its s3_path is NOT duplicated (keep regardless of status), OR
# - Its s3_path IS duplicated AND status = 1.0
mask_keep = (
    ~fin_stat_meta_df['s3_path'].isin(duplicate_s3_paths) |  # Not duplicated
    ((fin_stat_meta_df['s3_path'].isin(duplicate_s3_paths)) & (fin_stat_meta_df['status'] == 1.0))  # Duplicated but status = 1
)

# Step 3: Apply the mask to keep only the desired rows
fin_stat_meta_df_cleaned = fin_stat_meta_df[mask_keep].copy()

# Step 4: Check the results
print(f"Original rows: {len(fin_stat_meta_df)}")
print(f"Cleaned rows: {len(fin_stat_meta_df_cleaned)}")
print(f"Rows dropped: {len(fin_stat_meta_df) - len(fin_stat_meta_df_cleaned)}")

# Verify no duplicates remain (or only status=1.0 duplicates if they exist)
remaining_duplicates = fin_stat_meta_df_cleaned['s3_path'].value_counts()
remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]
print(f"\nRemaining duplicate s3_paths: {len(remaining_duplicates)}")
if len(remaining_duplicates) > 0:
    print("Note: Some s3_paths still have duplicates - checking their status:")
    for s3_path in remaining_duplicates.index[:5]:  # Show first 5
        statuses = fin_stat_meta_df_cleaned[fin_stat_meta_df_cleaned['s3_path'] == s3_path]['status'].unique()
        print(f"  {s3_path}: status values = {statuses}")

Original rows: 10113
Cleaned rows: 9153
Rows dropped: 960

Remaining duplicate s3_paths: 8
Note: Some s3_paths still have duplicates - checking their status:
  s3://jse-renamed-docs-copy/CSV-Copy/BIL/audited_financial_statements/2024/bil-barita_investments_limited_consolidated_statement_of_comprehensive_income-30-september-2024.csv: status values = [1.]
  s3://jse-renamed-docs-copy/CSV-Copy/MFS/unaudited_financial_statements/2023/mfs-MFS_capital_partners_limited_unaudited_consolidated_statement_of_financial_position-june-30-2023.csv: status values = [1.]
  s3://jse-renamed-docs-copy/CSV-Copy/CAC/audited_financial_statements/2022/cac-cac_2000_limited_statement_of_comprehensive_income-31-october-2022.csv: status values = [1.]
  s3://jse-renamed-docs-copy/CSV-Copy/KREMI/unaudited_financial_statements/2020/kremi-caribbean_cream_ltd_unaudited_income_statement-august-31-2020.csv: status values = [1.]
  s3://jse-renamed-docs-copy/CSV-Copy/BIL/unaudited_financial_statements/2020/bil-bartita_in

In [4]:
# Let's examine these remaining duplicates to see what differs between them
remaining_duplicate_paths = fin_stat_meta_df_cleaned['s3_path'].value_counts()
remaining_duplicate_paths = remaining_duplicate_paths[remaining_duplicate_paths > 1].index

print(f"Total rows with remaining duplicate s3_paths: {fin_stat_meta_df_cleaned['s3_path'].isin(remaining_duplicate_paths).sum()}")

# Show all the duplicate rows to see what's different
duplicate_rows = fin_stat_meta_df_cleaned[fin_stat_meta_df_cleaned['s3_path'].isin(remaining_duplicate_paths)]
duplicate_rows_sorted = duplicate_rows.sort_values('s3_path')
duplicate_rows_sorted.to_csv("remaining_duplicates.csv", index=False)

# Display key columns to understand the differences
print("\nDuplicate rows with status=1.0:")
print(duplicate_rows_sorted[['symbol', 's3_path', 'status', 'statement_type', 'period', 'period_end_date', 'report_type']].to_string())

Total rows with remaining duplicate s3_paths: 16

Duplicate rows with status=1.0:
      symbol                                                                                                                                                                               s3_path  status statement_type     period period_end_date       report_type
9064     BIL         s3://jse-renamed-docs-copy/CSV-Copy/BIL/audited_financial_statements/2022/bil-barita_investments_limited_consolidated_statement_of_comprehensive_income-30-september-2022.csv     1.0        audited     annual       30/9/2022     balance_sheet
9065     BIL         s3://jse-renamed-docs-copy/CSV-Copy/BIL/audited_financial_statements/2022/bil-barita_investments_limited_consolidated_statement_of_comprehensive_income-30-september-2022.csv     1.0        audited     annual       30/9/2022          cashflow
9069     BIL         s3://jse-renamed-docs-copy/CSV-Copy/BIL/audited_financial_statements/2024/bil-barita_investments_limited_con

In [5]:
# Manual mapping: s3_path -> correct statement_type to keep
# Review each file and update this mapping with the correct statement type
correct_statement_mapping = {
    # BIL files
    's3://jse-renamed-docs-copy/CSV-Copy/BIL/audited_financial_statements/2022/bil-barita_investments_limited_consolidated_statement_of_comprehensive_income-30-september-2022.csv': 'cashflow',  # Verified: contains cashflow
    's3://jse-renamed-docs-copy/CSV-Copy/BIL/audited_financial_statements/2024/bil-barita_investments_limited_consolidated_statement_of_comprehensive_income-30-september-2024.csv': 'income_statement',  # Verified: contains income statement
    's3://jse-renamed-docs-copy/CSV-Copy/BIL/unaudited_financial_statements/2020/bil-bartita_investments_limited_consolidated_statement_of_cash_flows-june-30-2020.csv': 'cashflow',  # TODO: verify
    
    # Other companies - TODO: review these files
    's3://jse-renamed-docs-copy/CSV-Copy/CAC/audited_financial_statements/2022/cac-cac_2000_limited_statement_of_comprehensive_income-31-october-2022.csv': 'income_statement',  # TODO: verify
    's3://jse-renamed-docs-copy/CSV-Copy/DTL/audited_financial_statements/2018/dtl-derrimon_trading_company_limited_group_statement_of_comprehensive_income-31-december-2018.csv': 'income_statement',  # TODO: verify
    's3://jse-renamed-docs-copy/CSV-Copy/JBG/unaudited_financial_statements/2023/jbg-jamaica_broilers_group_limited_group_statement_of_comprehensive_income-october-28-2023.csv': 'income_statement',  # TODO: verify
    's3://jse-renamed-docs-copy/CSV-Copy/KREMI/unaudited_financial_statements/2020/kremi-caribbean_cream_ltd_unaudited_income_statement-august-31-2020.csv': 'income_statement',  # TODO: verify
    's3://jse-renamed-docs-copy/CSV-Copy/MFS/unaudited_financial_statements/2023/mfs-MFS_capital_partners_limited_unaudited_consolidated_statement_of_financial_position-june-30-2023.csv': 'balance_sheet',  # TODO: verify
}

# Get list of duplicate s3_paths for reference
remaining_duplicate_paths = fin_stat_meta_df_cleaned['s3_path'].value_counts()
remaining_duplicate_paths = remaining_duplicate_paths[remaining_duplicate_paths > 1].index

print(f"Total duplicate s3_paths to review: {len(remaining_duplicate_paths)}")
print(f"Mapped so far: {len(correct_statement_mapping)}")
print(f"Still need to review: {len(remaining_duplicate_paths) - len(correct_statement_mapping)}")

# Apply the mapping
def should_keep_row(row):
    s3_path = row['s3_path']
    statement_type = row['statement_type']
    
    # If this s3_path is in our mapping, only keep if it matches the correct type
    if s3_path in correct_statement_mapping:
        return statement_type == correct_statement_mapping[s3_path]
    else:
        # Not in mapping (not a duplicate issue), keep it
        return True

# Apply the filter
mask = fin_stat_meta_df_cleaned.apply(should_keep_row, axis=1)
fin_stat_meta_df_final = fin_stat_meta_df_cleaned[mask].copy()

print(f"\nRows before: {len(fin_stat_meta_df_cleaned)}")
print(f"Rows after: {len(fin_stat_meta_df_final)}")
print(f"Rows dropped: {len(fin_stat_meta_df_cleaned) - len(fin_stat_meta_df_final)}")

# Verify no duplicates remain
remaining = fin_stat_meta_df_final['s3_path'].value_counts()
remaining_dups = remaining[remaining > 1]
print(f"\nRemaining duplicate s3_paths: {len(remaining_dups)}")

if len(remaining_dups) > 0:
    print("\nWARNING: Still have duplicates - need to add these to mapping:")
    for s3_path in remaining_dups.index:
        print(f"  - {s3_path}")

Total duplicate s3_paths to review: 8
Mapped so far: 8
Still need to review: 0

Rows before: 9153
Rows after: 9137
Rows dropped: 16

Remaining duplicate s3_paths: 0


In [ ]:

# First, identify which s3_paths are duplicates (appear more than once)
duplicate_s3_paths = fin_stat_meta_df['s3_path'].value_counts()
duplicate_s3_paths = duplicate_s3_paths[duplicate_s3_paths > 1].index
print(f"The number of duplicate s3_paths is {len(duplicate_s3_paths)}")

# Filter the dataframe to only rows with duplicate s3_paths
df_duplicates = fin_stat_meta_df[fin_stat_meta_df['s3_path'].isin(duplicate_s3_paths)]

# Count how many duplicate rows exist per status
duplicate_count_by_status = df_duplicates.groupby('status').size()
print(duplicate_count_by_status)
df_duplicates.to_csv("df_duplicates.csv", index=False)

# Count how many duplicate rows exist per statement_type
duplicate_count_by_statement_type = df_duplicates.groupby('statement_type').size()
print(duplicate_count_by_statement_type)

# Count how many duplicate rows exist per report_type


In [2]:
# Sort by the grouping columns and status (putting status=1 first)
# Then drop duplicates keeping the first occurrence (which will be status=1 if it exists)
fin_stat_meta_df_cleaned = (fin_stat_meta_df
    .sort_values(['symbol', 'period_end_date', 'report_type', 'consolidation_type', 'status'], 
                 na_position='last')  # This puts blank/NaN values last
    .drop_duplicates(subset=['symbol', 'period_end_date', 'report_type', 'consolidation_type'], 
                     keep='first')
)
fin_stat_meta_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8508 entries, 7 to 7794
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   symbol              8508 non-null   object 
 1   statement_type      8508 non-null   object 
 2   period              8508 non-null   object 
 3   period_detail       8508 non-null   object 
 4   period_end_date     8508 non-null   object 
 5   report_type         8508 non-null   object 
 6   consolidation_type  8508 non-null   object 
 7   status              1072 non-null   float64
 8   s3_path             7450 non-null   object 
 9   pdf_folder_path     8508 non-null   object 
 10  statement_ID        8508 non-null   object 
 11  pdf_s3_path         8348 non-null   object 
 12  is_missing          8508 non-null   bool   
 13  is_mismatched       8508 non-null   bool   
 14  is_incomplete       8508 non-null   bool   
 15  is_incomplete_note  859 non-null    object 
 16  is_mismatch

In [ ]:
counts = fin_stat_meta_df["s3_path"].value_counts()
duplicates = counts[counts > 1]
print(duplicates)
# Add a running count column to show the order of appearance

In [ ]:
# Get value counts and filter for counts > 1
counts = fin_stat_meta_df[["report_type", "statement_ID"]].value_counts()
duplicates = counts[counts > 1]
print(duplicates)

In [ ]:
# Add a running count column to show the order of appearance
fin_stat_meta_df['combination_order'] = fin_stat_meta_df.groupby([
    "symbol", "statement_type", "period", "period_detail", 
    "period_end_date", "report_type", "consolidation_type"
]).cumcount() + 1

# Now you can see the combinations with their order
result = fin_stat_meta_df[["symbol", "statement_type", "period", "period_detail", 
                          "period_end_date", "report_type", "consolidation_type", 
                          "combination_order"]].value_counts()
result

## PTL


In [ ]:
ptl = fin_stat_meta_df[fin_stat_meta_df["symbol"] == "PTL"]
ptl


In [ ]:
ptl_counts = ptl[[ "period_end_date", "report_type", "consolidation_type"]].value_counts()
duplicates = ptl_counts[ptl_counts > 1]
print(duplicates)


## 138SL

In [ ]:
symbol_138sl = fin_stat_meta_df[fin_stat_meta_df["symbol"] == "138SL"]
symbol_138sl

symbol_138sl_counts = symbol_138sl[[ "period_end_date", "report_type", "consolidation_type"]].value_counts()
duplicates = symbol_138sl_counts[symbol_138sl_counts > 1]
print(duplicates)
